In [1]:
%run ./config.py
import vectorbtpro as vbt 
import pandas as pd
import numpy as np

In [3]:
data = vbt.BinanceData.fetch(
    ['BTCUSDT', 'ETHUSDT'], 
    start='2020-01-01',
    end='2022-01-01',
    timeframe='1d'
)

  0%|          | 0/2 [00:00<?, ?it/s]

/Users/alvaroybanez/opt/anaconda3/envs/botchestra/lib/python3.9/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


0it [00:00, ?it/s]

/Users/alvaroybanez/opt/anaconda3/envs/botchestra/lib/python3.9/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


0it [00:00, ?it/s]

In [4]:
data.to_hdf('my_data.h5')

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [7]:
data = vbt.HDFData.fetch('my_data.h5')

  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
data.data['BTCUSDT'].info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17514 entries, 2019-12-31 23:00:00+00:00 to 2021-12-31 22:00:00+00:00
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   Open                17514 non-null  float64            
 1   High                17514 non-null  float64            
 2   Low                 17514 non-null  float64            
 3   Close               17514 non-null  float64            
 4   Volume              17514 non-null  float64            
 5   Close time          17514 non-null  datetime64[ns, UTC]
 6   Quote volume        17514 non-null  float64            
 7   Number of trades    17514 non-null  int64              
 8   Taker base volume   17514 non-null  float64            
 9   Taker quote volume  17514 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(8), int64(1)
memory usage: 2.0 MB


In [9]:
high = data.get('High')
low = data.get('Low')
close = data.get('Close')
close

symbol,BTCUSDT,ETHUSDT
Open time,,
2019-12-31 23:00:00+00:00,7195.23,129.16
2020-01-01 00:00:00+00:00,7177.02,128.87
2020-01-01 01:00:00+00:00,7216.27,130.64
2020-01-01 02:00:00+00:00,7242.85,130.85
2020-01-01 03:00:00+00:00,7225.01,130.20
...,...,...
2021-12-31 18:00:00+00:00,46686.41,3704.43
2021-12-31 19:00:00+00:00,45728.28,3626.27
2021-12-31 20:00:00+00:00,45879.24,3645.04


In [10]:
def get_med_price(high, low):
    return (high + low) / 2

def get_atr(high, low, close, period):
    tr0 = abs(high - low)
    tr1 = abs(high - close.shift())
    tr2 = abs(low - close.shift())
    tr = pd.concat((tr0, tr1, tr2), axis=1).max(axis=1)  
    atr = tr.ewm(
        alpha=1 / period, 
        adjust=False, 
        min_periods=period).mean()  
    return atr

def get_basic_bands(med_price, atr, multiplier):
    matr = multiplier * atr
    upper = med_price + matr
    lower = med_price - matr
    return upper, lower

def get_final_bands(close, upper, lower):  
    trend = pd.Series(np.full(close.shape, np.nan), index=close.index)
    dir_ = pd.Series(np.full(close.shape, 1), index=close.index)
    long = pd.Series(np.full(close.shape, np.nan), index=close.index)
    short = pd.Series(np.full(close.shape, np.nan), index=close.index)
    
    for i in range(1, close.shape[0]):  
        if close.iloc[i] > upper.iloc[i - 1]:
            dir_.iloc[i] = 1
        elif close.iloc[i] < lower.iloc[i - 1]:
            dir_.iloc[i] = -1
        else:
            dir_.iloc[i] = dir_.iloc[i - 1]
            if dir_.iloc[i] > 0 and lower.iloc[i] < lower.iloc[i - 1]:
                lower.iloc[i] = lower.iloc[i - 1]
            if dir_.iloc[i] < 0 and upper.iloc[i] > upper.iloc[i - 1]:
                upper.iloc[i] = upper.iloc[i - 1]

        if dir_.iloc[i] > 0:
            trend.iloc[i] = long.iloc[i] = lower.iloc[i]
        else:
            trend.iloc[i] = short.iloc[i] = upper.iloc[i]    
    return trend, dir_, long, short

def supertrend(high, low, close, period=7, multiplier=3):
    med_price = get_med_price(high, low)
    atr = get_atr(high, low, close, period)
    upper, lower = get_basic_bands(med_price, atr, multiplier)
    return get_final_bands(close, upper, lower)


In [11]:
supert, superd, superl, supers = supertrend(
    high['BTCUSDT'], 
    low['BTCUSDT'], 
    close['BTCUSDT']
)
supert


Open time
2019-12-31 23:00:00+00:00             NaN
2020-01-01 00:00:00+00:00             NaN
2020-01-01 01:00:00+00:00             NaN
2020-01-01 02:00:00+00:00             NaN
2020-01-01 03:00:00+00:00             NaN
                                 ...     
2021-12-31 18:00:00+00:00    48219.074906
2021-12-31 19:00:00+00:00    47858.398491
2021-12-31 20:00:00+00:00    47608.346563
2021-12-31 21:00:00+00:00    47608.346563
2021-12-31 22:00:00+00:00    47608.346563
Length: 17514, dtype: float64

In [12]:
date_range = slice('2020-01-01', '2020-02-01')
fig = close.loc[date_range, 'BTCUSDT'].rename('Close').vbt.plot()  
supers.loc[date_range].rename('Short').vbt.plot(fig=fig)
superl.loc[date_range].rename('Long').vbt.plot(fig=fig)

FigureWidget({
    'data': [{'name': 'Close',
              'showlegend': True,
              'type': 'scatter…